# Volatility Memory Analysis

<a id="index"></a>
## Index

[Setup on Kali 2020.2](#setup)

<a id="setup"></a>
## Setup on Kali 2020.2

[Back to Index](#index)

Steps to Install Volatility 
````
sudo apt update

sudo apt install -y python2 python-dev dwarfdump build-essential yara zip

curl https://bootstrap.pypa.io/get-pip.py --output get-pip.py

sudo python2 get-pip.py

sudo pip2 install setuptool pycrypto yara-python distorm3==3.4.4

git clone https://github.com/volatilityfoundation/volatility.git

cd volatility/

sudo python2 setup.py install
````

## Global Settings

In [9]:
directory='/home/kali/hackthebox/forensics'
#directory=!pwd
outputDir=directory + "/output"
dumpsDir=directory + "/dumps"
memFile=directory + "/" + "WIN-CEKM08E74HR-20150611-222930.raw"

!mkdir -p $outputDir
!mkdir -p $dumpsDir

## imageinfo 

In [6]:
!volatility -f $memFile imageinfo > $outputDir/imageinfo.txt
!cat $outputDir/imageinfo.txt

          Suggested Profile(s) : VistaSP1x86, Win2008SP1x86, Win2008SP2x86, VistaSP2x86
                     AS Layer1 : IA32PagedMemory (Kernel AS)
                     AS Layer2 : FileAddressSpace (/home/kali/hackthebox/forensics/WIN-CEKM08E74HR-20150611-222930.raw)
                      PAE type : No PAE
                           DTB : 0x122000L
                          KDBG : 0x8190ac98L
          Number of Processors : 1
     Image Type (Service Pack) : 2
                KPCR for CPU 0 : 0x8190b800L
             KUSER_SHARED_DATA : 0xffdf0000L
           Image date and time : 2015-06-11 22:29:32 UTC+0000
     Image local date and time : 2015-06-11 18:29:32 -0400


## Set Suggested Profile from imageinfo results

In [12]:
# Populate from the suggested profile from above
profile="VistaSP1x86"

## pslist

Provide a process list of what was running or had been running when the memory image was taken...

In [14]:
!volatility --profile=$profile -f $memFile pslist --output-file=$outputDir/pslist.txt
!cat $outputDir/pslist.txt

Volatility Foundation Volatility Framework 2.6.1
Outputting to: /home/kali/hackthebox/forensics/output/pslist.txt
Offset(V)  Name                    PID   PPID   Thds     Hnds   Sess  Wow64 Start                          Exit                          
---------- -------------------- ------ ------ ------ -------- ------ ------ ------------------------------ ------------------------------
0x8276d020 System                    4      0    100      507 ------      0 2015-06-11 22:05:22 UTC+0000                                 
0x8308dd90 smss.exe                400      4      4       28 ------      0 2015-06-11 22:05:22 UTC+0000                                 
0x82a35d90 csrss.exe               468    456     11      450      0      0 2015-06-11 22:05:26 UTC+0000                                 
0x830fd1b0 wininit.exe             520    456      3      100      0      0 2015-06-11 22:05:28 UTC+0000                                 
0x83139650 services.exe            596    520      6      

## psscan

Provide a process list based on found structure in memory.  If a process is hidding it will jump out of the flink and bliknk of the norm.

In [15]:
!volatility --profile=$profile -f $memFile psscan --output-file=$outputDir/psscan.txt
!cat $outputDir/psscan.txt

Volatility Foundation Volatility Framework 2.6.1
Outputting to: /home/kali/hackthebox/forensics/output/psscan.txt
Offset(P)          Name                PID   PPID PDB        Time created                   Time exited                   
------------------ ---------------- ------ ------ ---------- ------------------------------ ------------------------------
0x000000000276d020 System                4      0 0x00122000 2015-06-11 22:05:22 UTC+0000                                 
0x000000001ed913b8 taskeng.exe        2032   1012 0x0e292000 2015-06-11 22:05:59 UTC+0000                                 
0x000000001edeba10 cmd.exe            3604   1728 0x11705000 2015-06-11 22:25:15 UTC+0000   2015-06-11 22:25:15 UTC+0000  
0x000000001edf5718 cmd.exe            3520   1728 0x10a82000 2015-06-11 22:24:04 UTC+0000   2015-06-11 22:24:04 UTC+0000  
0x000000001ee15cd8 msdtc.exe           272    596 0x01509000 2015-06-11 22:08:10 UTC+0000                                 
0x000000001ee2fca0 bg.jpg

## pstree

In [16]:
!volatility --profile=$profile -f $memFile pstree --output-file=$outputDir/pstree.txt
!cat $outputDir/pstree.txt

Volatility Foundation Volatility Framework 2.6.1
Outputting to: /home/kali/hackthebox/forensics/output/pstree.txt
Name                                                  Pid   PPid   Thds   Hnds Time
-------------------------------------------------- ------ ------ ------ ------ ----
 0x837241d8:explorer.exe                             2428   2856     17    425 2015-06-11 22:27:18 UTC+0000
. 0x83201b90:Tomcat7w.exe                            2816   2428      2     50 2015-06-11 22:27:18 UTC+0000
. 0x83117020:jusched.exe                             3056   2428      1     50 2015-06-11 22:27:18 UTC+0000
. 0x8311ec10:cmd.exe                                 4032   2428      1     19 2015-06-11 22:29:10 UTC+0000
.. 0x830ecd90:DumpIt.exe                              744   4032      1     29 2015-06-11 22:29:30 UTC+0000
. 0x83632d08:VBoxTray.exe                            3060   2428      9    228 2015-06-11 22:27:18 UTC+0000
 0x830fd1b0:wininit.exe                               520    456      

## psxview

After the command executes search for processes that have false in the column for pslist and True in psscan.  This would mean that the process is hidding and not in the flink or blink of the process chain.

In [17]:
!volatility --profile=$profile -f $memFile psxview --output-file=$outputDir/psxview.txt
!cat $outputDir/psxview.txt

Volatility Foundation Volatility Framework 2.6.1
Outputting to: /home/kali/hackthebox/forensics/output/psxview.txt
Offset(P)  Name                    PID pslist psscan thrdproc pspcid csrss session deskthrd ExitTime
---------- -------------------- ------ ------ ------ -------- ------ ----- ------- -------- --------
0x1f140180 lsm.exe                 616 True   True   True     True   True  True    True     
0x1ef241d8 explorer.exe           2428 True   True   True     True   True  True    True     
0x1f1e5820 svchost.exe            1012 True   True   True     True   True  True    True     
0x1ee32d08 VBoxTray.exe           3060 True   True   True     True   True  True    True     
0x1f1d8d90 svchost.exe             968 True   True   True     True   True  True    True     
0x1ee15cd8 msdtc.exe               272 True   True   True     True   True  True    True     
0x1ee81610 dwm.exe                2860 True   True   True     True   True  True    True     
0x1f2beb98 svchost.exe          